<a href="https://colab.research.google.com/github/nurfnick/Data_Viz/blob/main/05_SQL_SubQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Sub-Queries

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


I am still gonna keep playing around with liquor data

In [2]:
%%bigquery --project pic-math
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 5

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-37362200037,2021-06-09,4582,Jiffy #926 / Spirit Lake,"2402, 17th St",Spirit Lake,51360,POINT (-95.126585 43.42282),30,DICKINSON,1092100,Imported Distilled Spirit Specialty,421.0,SAZERAC COMPANY INC,77487,Tortilla Gold DSS,12,1000,4.85,7.28,60,436.80,60.00,15.85
1,INV-21177600024,2019-08-12,2465,Sid's Beverage Shop,2727 Dodge St,Dubuque,52003,POINT (-90.705328 42.491862),31,DUBUQUE,1081100.0,Coffee Liqueurs,65,Jim Beam Brands,67557,Kamora Coffee Liqueur,12,1000,8.39,12.59,6,75.54,6.00,1.58
2,INV-25272100048,2020-02-17,2671,Hy-Vee / Jefferson,"106, W Washington St",Jefferson,50129.0,POINT (-94.375508 42.017267),37,GREENE,1081500.0,Triple Sec,434,LUXCO INC,86507,Paramount Triple Sec,12,1000,3.84,5.76,4,23.04,4.00,1.05
3,S27599300045,2015-08-31,2191,Keokuk Spirits,1013 MAIN,KEOKUK,52632,POINT (-91.387797 40.400038),56,Lee,1081700.0,DISTILLED SPIRITS SPECIALTY,434,Luxco-St Louis,75087,Juarez Gold Dss,12,1000,4.92,7.38,48,354.24,48.00,12.68
4,S21884700116,2014-10-20,2605,Hy-Vee Drugstore #5 / Cedar Rapids,2001 BLAIRS FERRY ROAD NE,CEDAR RAPIDS,52402,POINT (-91.668909 42.034799),57,Linn,1081010.0,AMERICAN AMARETTO,421,"Sazerac Co., Inc.",71886,Amaretto E Dolce,12,750,3.34,5.01,7,35.07,5.25,1.39


Let's ask what the most expensive bottle is in each category.

In [3]:
%%bigquery --project pic-math
SELECT category_name, MAX(state_bottle_retail) as max_state_bottle_retail
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY category_name

,category_name,f0_
0,Imported Distilled Spirit Specialty,342.99
1,Coffee Liqueurs,39.72
2,Triple Sec,18.50
3,DISTILLED SPIRITS SPECIALTY,112.01
4,AMERICAN AMARETTO,27.26
...,...,...
131,MISC. IMPORTED CORDIALS & LIQUEURS,658.19
132,Temporary & Specialty Packages,13.11
133,IMPORTED VODKA - CHERRY,10.58
134,American Whiskies,28.50


This was not too hard.  Let's me mean though and ask for the second most expensive bottle of "DISTILLED SPIRITS SPECIALTY".  Here is where the sub-query is going to come into play.  We don't want to include the most expensive so instead we remove those from the computation.

In [4]:
%%bigquery --project pic-math

SELECT MAX(state_bottle_retail) as max_state_bottle_retail
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE category_name = "DISTILLED SPIRITS SPECIALTY"



,max_state_bottle_retail
0,112.01


In [9]:
%%bigquery --project pic-math
SELECT MAX(state_bottle_retail) as max_state_bottle_retail
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE state_bottle_retail != (SELECT MAX(state_bottle_retail) as max_state_bottle_retail FROM `bigquery-public-data.iowa_liquor_sales.sales` WHERE category_name = "DISTILLED SPIRITS SPECIALTY")
      AND category_name = "DISTILLED SPIRITS SPECIALTY"

,max_state_bottle_retail
0,96.51
